In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

import random
random.seed(7)


def generator(X, Y, batch_size=32, train=True):
    while True:
        for offset in range(0, len(X), batch_size):
            X_batch = np.stack(X[offset:offset+batch_size], axis=0)
            Y_batch = np.stack(Y[offset:offset+batch_size], axis=0)
            
            Y_batch_ = np.empty((Y_batch.shape[0], 2,2,10))
            for m in range(Y_batch.shape[0]):
                for i in range(10):
                    Y_batch_[m, :,:,i] = [[np.sum(Y_batch[m, :5, :5, i]), np.sum(Y_batch[m, :5, 5:, i])], [np.sum(Y_batch[m, 5:, :5, i]), np.sum(Y_batch[m, 5:, 5:, i])]]
            Y_batch_[Y_batch_ > 0] = 1

            if train:
                Y_f = np.array([Y_batch_[e].flatten() for e in range(Y_batch_.shape[0])])
                yield (X_batch, Y_f)
            else:
                yield X_batch

structure_ids = []
for line in open('./structures lists/structures human.txt', 'r'):
    line = line.strip('\n')
    structure_ids.append(line)
# for line in open('./structures lists/structures ecoli.txt', 'r'):
#     line = line.strip('\n')
#     structure_ids.append(line)
structure_ids.remove('4pkd')
structure_ids.remove('1a9n')
structure_ids.remove('2adc')
random.shuffle(structure_ids)
print(len(structure_ids))

X_train = []
X_test = []
Y_train = []
Y_test = []
num_aa_train = 0
num_aa_test = 0
num_train = int(len(structure_ids)*0.7)
for i, structure_id in enumerate(structure_ids):
    protein = np.load('../data/voxelized data 10x10x10/' + structure_id + '_protein.npy', mmap_mode='r')
    rna = np.load('../data/voxelized data 10x10x10/' + structure_id + '_rna_3D.npy', mmap_mode='r')
    na = 0
    pos = 0
    while (np.sum(rna[na]) > 0) and (na < len(rna)-1):
        pos +=1
        na +=1
    

    if i <= num_train:
        X_train.extend(protein[:pos, :, :, :, :3])
        Y_train.extend(rna[:pos])
        num_aa_train +=pos
#         if pos > len(rna)/2:
#             X_train.extend(protein[:, :, :, :, :3])
#             Y_train.extend(rna)
#             num_aa_train +=len(rna)
#         else:
#             X_train.extend(protein[:pos, :, :, :, :3])
#             X_train.extend(protein[-pos:, :, :, :, :3])
#             Y_train.extend(rna[:pos])
#             Y_train.extend(rna[-pos:])
#             num_aa_train +=2*pos
    else:
        X_test.extend(protein[:pos, :, :, :, :3])
        Y_test.extend(rna[:pos])
        num_aa_test +=pos
#         if pos > len(rna)/2:
#             X_test.extend(protein[:, :, :, :, :3])
#             Y_test.extend(rna)
#             num_aa_test +=len(rna)
#         else:
#             X_test.extend(protein[:pos, :, :, :, :3])
#             X_test.extend(protein[-pos:, :, :, :, :3])
#             Y_test.extend(rna[:pos])
#             Y_test.extend(rna[-pos:])
#             num_aa_test +=2*pos

Y_test = np.stack(Y_test, axis=0)
Y_test_ = np.empty((len(Y_test), 2,2,10))
for m in range(len(Y_test)):
    for i in range(10):
        Y_test_[m, :,:,i] = [[np.sum(Y_test[m, :5, :5, i]), np.sum(Y_test[m, :5, 5:, i])], [np.sum(Y_test[m, 5:, :5, i]), np.sum(Y_test[m, 5:, 5:, i])]]
Y_test_prob = Y_test_
Y_test_[Y_test_ > 0] = 1
   
n_steps_train = int(num_aa_train/400) 
n_steps_test = int(num_aa_test/400)

print(num_aa_train, num_aa_test)

generator_train = generator(X_train, Y_train, 400, True)
generator_test = generator(X_test, Y_test, 400, False)

ins = tf.keras.layers.Input((10, 10, 10, 3))
con1 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu')(ins)
con2 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con1)
con3 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con2)
maxp1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con3)
con4 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp1)
con5 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con4)
con6 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con5)
maxp2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con6)
con7 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp2)
con8 = tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu')(con7)
con9 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu')(con8)
maxp3 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con9)
batch = tf.keras.layers.BatchNormalization()(maxp3)
flat = tf.keras.layers.Flatten()(batch)
dens2 = tf.keras.layers.Dense(units=256, activation='relu')(flat)
drop2 = tf.keras.layers.Dropout(0.6)(dens2)
outs = tf.keras.layers.Dense(units=40, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=ins, outputs=outs)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy', 'mse'])

model.summary()

# checkpoint
# filepath="weights_best.hdf5"
# checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
# callbacks_list = [checkpoint]

# model.fit(X_train, Y_train_f, validation_split=0.33, epochs=1, batch_size=200, callbacks=callbacks_list, verbose=0)
model.fit_generator(generator_train, steps_per_epoch=n_steps_train, epochs=500, callbacks=None, verbose=1, max_queue_size=2)

# model_best = model
# model_best.load_weights("weights_best.hdf5")
# print(model.evaluate(X_test, Y_test, verbose=0, batch_size=100))
# model_best.save('model_cnn_15_2.h5')
# Y_pred = model_best.predict(X_test, batch_size=200)
Y_pred = model.predict_generator(generator_test, steps=n_steps_test)
print(Y_pred.shape)
Y_pred_ = np.array([Y_pred[i].reshape((2,2,10)) for i in range(Y_pred.shape[0])])

#CNN
Y_pred_prob = Y_pred_
Y_pred_[Y_pred_ >= 0.5] = 1
Y_pred_[Y_pred_ < 0.5] = 0

print(Y_pred_.shape)
print(Y_test_.shape)
Y_test_ = Y_test_[:Y_pred_.shape[0]]
Y_test_prob = Y_test_prob[:Y_pred_.shape[0]]

print('CNN: \n')
for i in range(10):
    confusion_matrix = [sklearn.metrics.confusion_matrix(Y_test_[:,l , c, i], Y_pred_[:,l , c, i]) for l in range(2) for c in range(2)]
    accuracy = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix]
    auc = [sklearn.metrics.roc_auc_score(Y_test_prob[:,l , c, i], Y_pred_prob[:,l , c, i]) for l in range(2) for c in range(2)]

    print(f'level {i}')
    for q in range(len(confusion_matrix)):
        print(confusion_matrix[q], np.round(accuracy[q], 2), np.round(auc[q], 2))

# baseline model
# predict all zeros; at least 50% correct predictions because there are 1/2 of negative examples
Y_pred_base = np.zeros(Y_test_.shape)

# po = np.sum(Y_train, axis=0)/Y_train.shape[0]
# po[po >= 0.5] = 1
# po[po < 0.5] = 0
# Y_pred_base = np.tile(po, (Y_test.shape[0],1))

print(Y_pred_base.shape)
print(f'\n BASELINE MODEL: \n')
for i in range(10):
    confusion_matrix_base = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]
    accuracy_base = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_base]
    auc_base = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_base[:,l, c, i]) for l in range(2) for c in range(2)]
    
    print(f'level {i}')
    for q in range(len(confusion_matrix_base)):
        print(confusion_matrix_base[q], np.round(accuracy_base[q], 2), np.round(auc_base[q], 2))

#random model
Y_pred_random = np.random.random(Y_test_.shape)
Y_pred_random[Y_pred_random >= 0.5] = 1
Y_pred_random[Y_pred_random < 0.5] = 0

print(f'\n RANDOM MODEL: \n')
for i in range(10):
    confusion_matrix_random = [sklearn.metrics.confusion_matrix(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]
    accuracy_random = [np.sum(np.trace(cm))/np.sum(cm) for cm in confusion_matrix_random]
    auc_random = [sklearn.metrics.roc_auc_score(Y_test_[:,l, c, i], Y_pred_random[:,l, c, i]) for l in range(2) for c in range(2)]

    print(f'level {i}')
    for q in range(len(confusion_matrix_random)):
        print(confusion_matrix_random[q], np.round(accuracy_random[q], 2), np.round(auc_random[q], 2))

230
51901 27760
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10, 10, 10, 3)     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 10, 10, 10, 64)    5248      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 10, 10, 10, 32)    55328     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 10, 10, 10, 32)    27680     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 5, 5, 5, 32)       0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 5, 5, 5, 32)       27680     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 5, 5, 5, 16)       13840

129/129 [==============================] - 18s 138ms/step - loss: 0.5455 - acc: 0.7620 - mean_squared_error: 0.1805
Epoch 44/500
129/129 [==============================] - 17s 135ms/step - loss: 0.5440 - acc: 0.7626 - mean_squared_error: 0.1798
Epoch 45/500
129/129 [==============================] - 17s 133ms/step - loss: 0.5434 - acc: 0.7624 - mean_squared_error: 0.1796
Epoch 46/500
129/129 [==============================] - 17s 133ms/step - loss: 0.5419 - acc: 0.7625 - mean_squared_error: 0.1790
Epoch 47/500
129/129 [==============================] - 18s 142ms/step - loss: 0.5419 - acc: 0.7620 - mean_squared_error: 0.1790
Epoch 48/500
129/129 [==============================] - 18s 139ms/step - loss: 0.5399 - acc: 0.7631 - mean_squared_error: 0.1781
Epoch 49/500
129/129 [==============================] - 18s 141ms/step - loss: 0.5391 - acc: 0.7635 - mean_squared_error: 0.1778
Epoch 50/500
129/129 [==============================] - 18s 137ms/step - loss: 0.5388 - acc: 0.7626 - mean_squ

129/129 [==============================] - 18s 138ms/step - loss: 0.5184 - acc: 0.7665 - mean_squared_error: 0.1699
Epoch 107/500
129/129 [==============================] - 19s 148ms/step - loss: 0.5205 - acc: 0.7636 - mean_squared_error: 0.1709
Epoch 108/500
129/129 [==============================] - 17s 135ms/step - loss: 0.5205 - acc: 0.7633 - mean_squared_error: 0.1709
Epoch 109/500
129/129 [==============================] - 18s 137ms/step - loss: 0.5201 - acc: 0.7634 - mean_squared_error: 0.1708
Epoch 110/500
129/129 [==============================] - 18s 138ms/step - loss: 0.5202 - acc: 0.7633 - mean_squared_error: 0.1708
Epoch 111/500
129/129 [==============================] - 18s 138ms/step - loss: 0.5196 - acc: 0.7638 - mean_squared_error: 0.1705
Epoch 112/500
129/129 [==============================] - 19s 147ms/step - loss: 0.5196 - acc: 0.7633 - mean_squared_error: 0.1706
Epoch 113/500
129/129 [==============================] - 17s 132ms/step - loss: 0.5170 - acc: 0.7654 - m

129/129 [==============================] - 17s 134ms/step - loss: 0.5044 - acc: 0.7637 - mean_squared_error: 0.1656
Epoch 170/500
129/129 [==============================] - 18s 140ms/step - loss: 0.5030 - acc: 0.7647 - mean_squared_error: 0.1650
Epoch 171/500
129/129 [==============================] - 18s 139ms/step - loss: 0.5018 - acc: 0.7656 - mean_squared_error: 0.1645
Epoch 172/500
129/129 [==============================] - 18s 140ms/step - loss: 0.5009 - acc: 0.7667 - mean_squared_error: 0.1642
Epoch 173/500
129/129 [==============================] - 19s 146ms/step - loss: 0.5026 - acc: 0.7641 - mean_squared_error: 0.1649
Epoch 174/500
129/129 [==============================] - 18s 139ms/step - loss: 0.5016 - acc: 0.7645 - mean_squared_error: 0.1646
Epoch 175/500
129/129 [==============================] - 18s 139ms/step - loss: 0.5019 - acc: 0.7643 - mean_squared_error: 0.1647
Epoch 176/500
129/129 [==============================] - 19s 149ms/step - loss: 0.5014 - acc: 0.7644 - m

Epoch 232/500
129/129 [==============================] - 20s 152ms/step - loss: 0.4879 - acc: 0.7652 - mean_squared_error: 0.1600
Epoch 233/500
129/129 [==============================] - 18s 136ms/step - loss: 0.4870 - acc: 0.7660 - mean_squared_error: 0.1596
Epoch 234/500
129/129 [==============================] - 18s 142ms/step - loss: 0.4860 - acc: 0.7667 - mean_squared_error: 0.1592
Epoch 235/500
129/129 [==============================] - 18s 140ms/step - loss: 0.4845 - acc: 0.7680 - mean_squared_error: 0.1586
Epoch 236/500
129/129 [==============================] - 18s 138ms/step - loss: 0.4844 - acc: 0.7686 - mean_squared_error: 0.1585
Epoch 237/500
129/129 [==============================] - 18s 139ms/step - loss: 0.4863 - acc: 0.7657 - mean_squared_error: 0.1594
Epoch 238/500
129/129 [==============================] - 18s 141ms/step - loss: 0.4861 - acc: 0.7655 - mean_squared_error: 0.1594
Epoch 239/500
129/129 [==============================] - 19s 144ms/step - loss: 0.4860 - a

129/129 [==============================] - 18s 138ms/step - loss: 0.4752 - acc: 0.7681 - mean_squared_error: 0.1555
Epoch 295/500
129/129 [==============================] - 17s 132ms/step - loss: 0.4758 - acc: 0.7678 - mean_squared_error: 0.1558
Epoch 296/500
129/129 [==============================] - 18s 137ms/step - loss: 0.4746 - acc: 0.7686 - mean_squared_error: 0.1553
Epoch 297/500
129/129 [==============================] - 17s 132ms/step - loss: 0.4741 - acc: 0.7692 - mean_squared_error: 0.1551
Epoch 298/500
129/129 [==============================] - 18s 142ms/step - loss: 0.4742 - acc: 0.7682 - mean_squared_error: 0.1552
Epoch 299/500
129/129 [==============================] - 18s 141ms/step - loss: 0.4756 - acc: 0.7680 - mean_squared_error: 0.1557
Epoch 300/500
129/129 [==============================] - 18s 138ms/step - loss: 0.4744 - acc: 0.7692 - mean_squared_error: 0.1552
Epoch 301/500
129/129 [==============================] - 18s 139ms/step - loss: 0.4730 - acc: 0.7698 - m

129/129 [==============================] - 18s 139ms/step - loss: 0.4620 - acc: 0.7750 - mean_squared_error: 0.1507
Epoch 358/500
129/129 [==============================] - 19s 144ms/step - loss: 0.4641 - acc: 0.7722 - mean_squared_error: 0.1517
Epoch 359/500
129/129 [==============================] - 18s 137ms/step - loss: 0.4640 - acc: 0.7727 - mean_squared_error: 0.1516
Epoch 360/500
129/129 [==============================] - 18s 140ms/step - loss: 0.4637 - acc: 0.7732 - mean_squared_error: 0.1515
Epoch 361/500
129/129 [==============================] - 19s 145ms/step - loss: 0.4647 - acc: 0.7723 - mean_squared_error: 0.1519
Epoch 362/500
129/129 [==============================] - 17s 130ms/step - loss: 0.4642 - acc: 0.7728 - mean_squared_error: 0.1517
Epoch 363/500
129/129 [==============================] - 17s 132ms/step - loss: 0.4633 - acc: 0.7733 - mean_squared_error: 0.1513
Epoch 364/500
129/129 [==============================] - 17s 134ms/step - loss: 0.4623 - acc: 0.7744 - m

Epoch 420/500
129/129 [==============================] - 18s 138ms/step - loss: 0.4536 - acc: 0.7799 - mean_squared_error: 0.1477
Epoch 421/500
129/129 [==============================] - 19s 145ms/step - loss: 0.4552 - acc: 0.7783 - mean_squared_error: 0.1484
Epoch 422/500
129/129 [==============================] - 18s 141ms/step - loss: 0.4540 - acc: 0.7791 - mean_squared_error: 0.1479
Epoch 423/500
129/129 [==============================] - 19s 144ms/step - loss: 0.4542 - acc: 0.7791 - mean_squared_error: 0.1480
Epoch 424/500
129/129 [==============================] - 18s 137ms/step - loss: 0.4538 - acc: 0.7793 - mean_squared_error: 0.1479
Epoch 425/500
129/129 [==============================] - 18s 138ms/step - loss: 0.4545 - acc: 0.7793 - mean_squared_error: 0.1481
Epoch 426/500
129/129 [==============================] - 19s 143ms/step - loss: 0.4531 - acc: 0.7801 - mean_squared_error: 0.1476
Epoch 427/500
129/129 [==============================] - 18s 143ms/step - loss: 0.4529 - a

129/129 [==============================] - 18s 138ms/step - loss: 0.4468 - acc: 0.7860 - mean_squared_error: 0.1451
Epoch 483/500
129/129 [==============================] - 17s 133ms/step - loss: 0.4460 - acc: 0.7866 - mean_squared_error: 0.1448
Epoch 484/500
129/129 [==============================] - 17s 130ms/step - loss: 0.4460 - acc: 0.7865 - mean_squared_error: 0.1448
Epoch 485/500
129/129 [==============================] - 18s 143ms/step - loss: 0.4473 - acc: 0.7853 - mean_squared_error: 0.1454
Epoch 486/500
129/129 [==============================] - 18s 138ms/step - loss: 0.4461 - acc: 0.7861 - mean_squared_error: 0.1449
Epoch 487/500
129/129 [==============================] - 18s 139ms/step - loss: 0.4444 - acc: 0.7879 - mean_squared_error: 0.1442
Epoch 488/500
129/129 [==============================] - 18s 140ms/step - loss: 0.4465 - acc: 0.7859 - mean_squared_error: 0.1451
Epoch 489/500
129/129 [==============================] - 17s 131ms/step - loss: 0.4462 - acc: 0.7861 - m